In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies.buy_x_week_low)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col('Date') >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
sma100 = pi.indicators.simple_moving_average(df, 100)
df = sma100.df
slope_up = pi.indicators.slope_up(df, column=sma100.column)
df = slope_up.df.with_columns(pl.col(slope_up.column).shift()) #shifting so we use previous bar

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
lookback = timedelta(weeks=52)
modresult = pi.strategies.buy_x_week_low.strategy(df, lookback)

In [ ]:
trades = pi.summarize.trades.Trades.from_strategy_result(modresult, [slope_up.column])
trades.df

In [ ]:
trades.summarize_strategy(slope_up.column)